In [16]:
from src.graph import graph_loader, create_polarized_graph, spectral_bipartition_coloring, random_color_graph
from src.seed import seed_degree, seed_random 
import networkx as nx
from icm_diffusion import simulate_diffusion_ICM
import pandas as pd
import random

In [17]:
G = create_polarized_graph(500, 0.2, 0.01)
#color the graph
spectral_bipartition_coloring(G)

/Users/quenzer/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:233: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [20]:
def edge_addition_adamic_adar(G, seeds, k):
    graph = G.copy()

    # Convert to undirected graph for Adamic-Adar calculation
    undirected_graph = graph.to_undirected()

    for seed in seeds:
        # Compute Adamic-Adar index for pairs involving the seed node
        adamic_adar_scores = list(nx.adamic_adar_index(undirected_graph, [(seed, n) for n in undirected_graph.nodes if n != seed]))

        # Sort the scores in descending order
        adamic_adar_scores.sort(key=lambda x: x[2], reverse=True)

        # Add edges to the graph as a connection from the seed to the top k nodes
        for i in range(min(k, len(adamic_adar_scores))):
            target_node = adamic_adar_scores[i][1]
            graph.add_edge(seed, target_node)

    return graph

In [21]:
def edge_addition_preferential_attachment(G, seeds, k):
    graph = G.copy()
    
    for seed in seeds:
        # Calculate the degree of all nodes in the graph
        node_degrees = dict(graph.degree())
        
        # Generate the cumulative distribution of degrees for random selection
        nodes, degrees = zip(*node_degrees.items())
        total_degree = sum(degrees)
        cumulative_distribution = [sum(degrees[:i+1]) / total_degree for i in range(len(degrees))]

        # Add edges from the seed node to k nodes chosen by the preferential attachment rule
        for _ in range(k):
            random_value = random.random()
            for i, cum_dist in enumerate(cumulative_distribution):
                if random_value <= cum_dist:
                    target_node = nodes[i]
                    # Prevent self-loops and duplicate edges
                    if target_node != seed and not graph.has_edge(seed, target_node):
                        graph.add_edge(seed, target_node)
                        break
    
    return graph

In [ ]:
seed = seed_degree(G, 50)
k = 15

In [22]:
# Simulate diffusion on the original graph
count, count_std, color_count, color_count_std = simulate_diffusion_ICM(G, seed, 1, 1000)

# Results for the original graph
original_results = pd.DataFrame({
    'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
    'Original Graph': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
})

# Simulate diffusion on the adapted graph
G_adamic_adar = edge_addition_adamic_adar(G, seed, k)
count, count_std, color_count, color_count_std = simulate_diffusion_ICM(G_adamic_adar, seed, 1, 1000)

# Results for the adapted graph
adapted_results = pd.DataFrame({
    'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
    'Adapted Graph Adar': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
})

# Combine results into a single DataFrame
combined_results = pd.merge(original_results, adapted_results, on='Metric')

# Simulate diffusion on the adapted graph
G_PrefAtt = edge_addition_preferential_attachment(G, seed, k)
count, count_std, color_count, color_count_std = simulate_diffusion_ICM(G_PrefAtt, seed, 1, 1000)

# Results for the adapted graph
adapted_results = pd.DataFrame({
    'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
    'Adapted Graph PrefAtt': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
})

# Combine results into a single DataFrame
combined_results = pd.merge(combined_results, adapted_results, on='Metric')


# Get the number of nodes and edges for both graphs
original_graph_info = {
    'Metric': ['Number of Nodes', 'Number of Edges'],
    'Original Graph': [G.number_of_nodes(), G.number_of_edges()],
    'Adapted Graph Adar': [G_adamic_adar.number_of_nodes(), G_adamic_adar.number_of_edges()],
    'Adapted Graph PrefAtt': [G_PrefAtt.number_of_nodes(), G_PrefAtt.number_of_edges()]
}

graph_info_df = pd.DataFrame(original_graph_info)

# Combine all results into one DataFrame
final_results = pd.concat([graph_info_df, combined_results], ignore_index=True)

# Transpose the DataFrame and set the first row as the header
final_results = final_results.T
final_results.columns = final_results.iloc[0]  # Set the first row as the column names
final_results = final_results.drop(final_results.index[0])  # Drop the first row


100%|██████████| 2000/2000 [00:06<00:00, 319.66it/s]


In [23]:
final_results

Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,500.0,25573.0,137.672,79.175,3.827,5.357
Adapted Graph Adar,500.0,26178.0,143.91,78.953,3.719,5.189
Adapted Graph PrefAtt,500.0,26323.0,150.444,79.082,10.094,12.179
